Generation of Question with zero shot learning from Vicuna 13B

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import pandas as pd
# from datasets import load_dataset
import torch
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from evaluate import load
import numpy as np
import itertools
import sys
	
import time

2024-05-03 18:52:51.868961: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 18:52:51.910235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 18:52:52.499523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

device = torch.device("cuda:1")
torch.cuda.set_device(device)
print(device)

In [2]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    df_multi = []
    df_phrase = []
    df_passage = []
    with open('/DATA1/ai20btech11028/vojes_nlp/Data/' + file_name, encoding='utf-8') as f:
        for i in f:
            i = json.loads(i)
                
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            spoilers = i['spoiler']
            label = i['tags']
            keywords = i['targetKeywords']
            positions = i['spoilerPositions']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'phrase' or label == 'phrases':
                label = 0
            elif label == 'multi':
                label = 2
            else:
                label = 1
                
            
            df += [{'question': tweet , 'context': i['targetParagraphs'], 'article': article_title + article, 'spoiler':spoilers,
                    'labels': label, 'keywords':keywords, 'spoilerPositions':positions}]

    return pd.DataFrame(df)
            
    
# # test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
# print(train_dataset)

In [3]:
train_dataset

,question,context,article,spoiler,labels,keywords,spoilerPositions
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?],1,"new england patriots, ricky doyle, top stories,","[[[3, 151], [3, 186]]]"
1,NASA sets date for full recovery of ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,[2070],0,"ozone layer,ozone hole determined by weather,M...","[[[0, 0], [0, 4]]]"
2,This is what makes employees happy -- and it's...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation],0,"employee happiness money,employee happiness in...","[[[1, 186], [1, 210]]]"
3,Passion is overrated — 7 work habits you need ...,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...,2,"business, work-life, careers","[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ..."
4,The perfect way to cook rice so that it's perf...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,[in a rice cooker],0,"Quora,users,share,perfect,way,cook,rice","[[[5, 60], [5, 76]]]"
...,...,...,...,...,...,...,...
3195,Has Facebook's video explosion completely shak...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",[it hasn’t necessarily taken the wind out of Y...,1,"Facebook,web video,web video ads,YouTube","[[[7, 50], [7, 118]]]"
3196,Cop Is Eating At A Chili's When Teen Hands Him...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,"[It read, ""Thanks for keeping us safe.""]",1,None,"[[[0, 317], [0, 355]]]"
3197,5 popular myths about visible signs of aging t...,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,[1. Anti-wrinkle creams will erase the fine li...,2,,"[[[6, 0], [6, 73]], [[10, 0], [10, 109]], [[14..."
3198,You need to see this Twitter account that pred...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",[@beyoncefan666],0,,"[[[3, 55], [3, 69]]]"


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")
model = AutoModelForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
%%time
PROMPT =(
        "For the below clickbait sentence from which write a interrogative questions to extract answer from the news article\n"
        "### Sentence:\n{question}\n### Question:" )


sample = train_dataset.iloc[5]
clickbait = sample['question']
input_text = PROMPT.format(question=clickbait)

input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Below is a clickbait sentence from which write a interrogative questions to extract answer from the news article
### Sentence:
What happens if your new AirPods get lost or stolen, will Apple do anything?
### Question:
What is the policy of Apple when it comes to replacing lost or stolen AirPods?
CPU times: user 10min 46s, sys: 338 ms, total: 10min 47s
Wall time: 20.5 s


In [ ]:
generated_questions = []
sen_ques = re.compile(r"Question:\n(.*)\n?", re.MULTILINE)
j = 0
for i in tqdm(range(len(clickbaits) // 50)):
    clickbaits_batch = clickbaits[i * 50 : (i + 1) * 50
    
    input_ids = tokenizer([PROMPT.format(question=c) for c in clickbaits_batch], padding=True, return_tensors="pt")
    with torch.inference_mode():
        generated_ids = model.generate(**input_ids, max_new_tokens=30)

    questions_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    for question in tqdm(questions_batch):
        try:
            generated = next(sen_ques.finditer(question)).groups(0)[0]
        except StopIteration:
            generated = clickbaits[j]
        j += 1
        generated_questions.append(generated)